## 載入套件

In [4]:
from __future__ import print_function
import keras
import tensorflow as tf 
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils


## 確認是否有GPU

In [5]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce RTX 3080 Laptop GPU


In [3]:
# !nvidia-smi
!nvidia-smi

'nvidia-smi' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [7]:
60000/128

468.75

In [8]:
28*28

784

## 設定參數

In [6]:
# 定義梯度下降批量
batch_size = 128

# 定義分類數量, 0~9 總共 10 個
num_classes = 10

# 定義訓練週期, 4 次輪迴計算
epochs = 4

# 定義圖像寬、高
img_rows, img_cols = 28, 28



## 資料放置的方式:

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 保留原始資料，供 cross tab function 使用
y_test_org = y_test

# channels_first: 樣本數在第一維度, 色彩通道(R/G/B)資料(深度)放在第2維度，第3、4維度放置寬與高

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else: # channels_last: 樣本數在第一維度, 第2、3維度放置寬與高, 色彩通道(R/G/B)資料(深度)放在第4維度
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [8]:
print(x_train.shape)

(60000, 28, 28, 1)


## 轉換資料集

In [9]:
# 轉換明暗度 0~255 資料為 0~1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# y 值轉成 one-hot encoding
# 因為 y 值是 0~9, 所以有 10 個類別, 要分類, 但分類不能有級別, 所以要轉成 one-hot 
# encoding, 免得權重失衡 (數字大的, 變得比較重要)

#y_train = keras.utils.to_categorical(y_train, num_classes)
y_train = np_utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)


In [10]:
print(y_test[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [11]:
print(y_train.shape)
print(y_train[0:5])

(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 建立 model

In [12]:
# 建立簡單的線性執行的模型
model = Sequential()
# 建立卷積層，filter=32,即 output space 的深度, Kernal Size: 3x3, activation function 採用 relu
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
# 建立卷積層，filter=64,即 output size, Kernal Size: 3x3, activation function 採用 relu
model.add(Conv2D(64, (3, 3), activation='relu'))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))
# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))
# Flatten層把多維的輸入一維化，常用在從卷積層到全連接層的過渡。
model.add(Flatten())
# 全連接層: 128個output
model.add(Dense(128, activation='relu'))
# Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.5
model.add(Dropout(0.5))
# 使用 softmax activation function，將結果分類
model.add(Dense(num_classes, activation='softmax'))

# 編譯: 選擇損失函數、優化方法及成效衡量方式
model.compile(loss=keras.losses.categorical_crossentropy,
              #optimizer=keras.optimizers.Adadelta(),
              #optimizer=keras.optimizers.Adam(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# 進行訓練, 訓練過程會存在 train_history 變數中
train_history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# 顯示損失函數、訓練成果(分數)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/4
469/469 [==============================] - 6s 5ms/step - loss: 0.2454 - accuracy: 0.9238 - val_loss: 0.0526 - val_accuracy: 0.9841
Epoch 2/4
469/469 [==============================] - 2s 4ms/step - loss: 0.0838 - accuracy: 0.9756 - val_loss: 0.0416 - val_accuracy: 0.9872
Epoch 3/4
469/469 [==============================] - 2s 4ms/step - loss: 0.0644 - accuracy: 0.9801 - val_loss: 0.0350 - val_accuracy: 0.9886
Epoch 4/4
469/469 [==============================] - 2s 4ms/step - loss: 0.0517 - accuracy: 0.9836 - val_loss: 0.0293 - val_accuracy: 0.9911
Test loss: 0.029326092451810837
Test accuracy: 0.991100013256073


## 混淆矩陣

In [13]:
# 計算『混淆矩陣』(Confusion Matrix)，顯示測試集分類的正確及錯認總和數
import pandas as pd 
import numpy as np
from sklearn.metrics import classification_report
#predictions = model.predict_classes(x_test) 
predictions = model.predict(x_test) 

predictions = np.argmax(predictions,axis=1)
pd.crosstab(y_test_org, predictions, rownames=['實際值'], colnames=['預測值'])



313/313 [==============================] - 1s 2ms/step


預測值,0,1,2,3,4,5,6,7,8,9
實際值,,,,,,,,,,
0,973,1,1,1,0,1,2,0,1,0
1,0,1131,1,0,0,1,2,0,0,0
2,1,3,1022,2,0,0,0,2,2,0
3,0,0,1,1005,0,2,0,1,1,0
4,0,0,0,0,978,0,1,0,1,2
5,0,0,1,3,0,887,1,0,0,0
6,3,2,0,0,2,3,946,0,2,0
7,0,2,6,2,0,0,0,1015,2,1
8,1,0,1,1,0,2,1,3,964,1
